In [40]:
import scipy
import tensorflow
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, LSTM, Reshape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import os
import random
import matplotlib.pyplot as plt
import pandas as pd

from PyEMD import EMD


In [2]:
X = np.random.randn(1000, 100, 1)  # Shape: (samples, time points, channels)
y = np.random.randint(0, 2, (1000,))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
# Fit on training set only
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
# Apply transform to both the training set and the test set
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [3]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_split=0.2)

_, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/10


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5301 - loss: 0.7219 - val_accuracy: 0.4688 - val_loss: 0.7017
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5956 - loss: 0.6697 - val_accuracy: 0.4625 - val_loss: 0.7686
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5678 - loss: 0.6740 - val_accuracy: 0.4375 - val_loss: 0.7575
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6588 - loss: 0.6356 - val_accuracy: 0.5125 - val_loss: 0.7252
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6760 - loss: 0.6221 - val_accuracy: 0.4563 - val_loss: 0.7433
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6892 - loss: 0.5920 - val_accuracy: 0.4500 - val_loss: 0.7816
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7462 - loss: 0.5643 - val_accuracy: 0.4437 - val_loss: 0.7984
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7368 - loss: 0.5526 - val_accuracy: 0.4250 - val_loss: 0.8545
Epo

In [2]:
# dataset_dir_control = "csv_averaged_force_data_4col/GaCo04_01.csv"
# dataset_dir_pd = "csv_averaged_force_data_4col/GaPt03_01.csv"

features = ['Time','VGRF_L1','VGRF_L2','VGRF_L3','VGRF_L4','VGRF_L5','VGRF_L6','VGRF_L7','VGRF_L8',
            'VGRF_R1','VGRF_R2','VGRF_R3','VGRF_R4','VGRF_R5','VGRF_R6','VGRF_R7','VGRF_R8','Total Force_L','Total Force_R']
file = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Ga_study/GaCo01_01.txt"
df = pd.read_csv(file, delim_whitespace=True, header=None)
df.columns = features


In [3]:
# Calcuating the FAV = |L_k - R_k|/L_k  x 100%
# FAV is the fluctuation amplitude variability defined as a criterion in the paper
data_directory_Ga = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Ga_study"
data_directory_Ju = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Ju_study"
data_directory_Si = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Si_study"
# Defining a function that goes through the entire sensor column and finding the FAV
def calculate_fav(df):
    fav_values = {}

    for i in range(1, 9):
        L = df[f'VGRF_L{i}']
        R = df[f'VGRF_R{i}']
        # Filter out instances where L is near-zero to avoid distortion in FAV calculation
        valid_indices = L > 0.01  # or another threshold appropriate for your data
        L_valid = L[valid_indices]
        R_valid = R[valid_indices]
        
        if not L_valid.empty:  # Ensure there are valid data points for calculation
            fav = (abs(L_valid - R_valid) / L_valid).mean() * 100
            fav_values[i] = fav
        else:
            fav_values[i] = 0  # or set to 0 or other indicator value

    # Sort sensors by FAV in descending order, excluding None values
    sorted_fav = sorted(fav_values.items(), key=lambda item: item[1], reverse=True)
    
    top_sensors = sorted_fav[:4]
    return top_sensors

In [4]:
top_sensors = calculate_fav(df)
print("Top 4 sensors based on FAV:", top_sensors)

Top 4 sensors based on FAV: [(1, 604.5618947398632), (7, 567.9507063523395), (6, 343.4762884711009), (4, 329.90535681168546)]


In [5]:
## Selecting the optimal sensors for each file and making a csv file 
all_fav_results = {}
data_files = [f for f in os.listdir(data_directory_Ju) if os.path.isfile(os.path.join(data_directory_Ju, f))]
features = ['Time','VGRF_L1','VGRF_L2','VGRF_L3','VGRF_L4','VGRF_L5','VGRF_L6','VGRF_L7','VGRF_L8',
            'VGRF_R1','VGRF_R2','VGRF_R3','VGRF_R4','VGRF_R5','VGRF_R6','VGRF_R7','VGRF_R8','Total Force_L','Total Force_R']

file_error = []
for file_name in data_files:
    if file_name.startswith('.'):
        # Skip hidden files like .DS_Store
        continue
    
    file_path = os.path.join(data_directory_Ju, file_name)

    try:
        df = pd.read_csv(file_path, delim_whitespace=True, header=None, encoding_errors = 'ignore')
        df.columns = ['Time'] + [f'VGRF_L{i}' for i in range(1, 9)] + [f'VGRF_R{i}' for i in range(1, 9)] + ['Total_L', 'Total_R']

        top_sensors = calculate_fav(df)
        all_fav_results[file_name] = top_sensors
    except Exception as e:
        file_error.append(file_path)


In [6]:
print(file_error)

[]


In [7]:
# Assuming `fav_values` is a dictionary with your FAV data
fav_df = pd.DataFrame(list(all_fav_results.items()), columns=['File_name', 'Optimal 4 Sensor numbers to FAV Dictionary'])
fav_df.to_csv('fav_values_Ju.csv', index=False)

In [42]:
# Selected Sensors: 2, 4, 6, 8
file = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Ga_study/GaCo03_02.txt"
df = pd.read_csv(file, delim_whitespace=True, header=None)
df.columns = features


def find_imfs(df):
    selected_sensors = [2, 4, 6, 8]
    emd = EMD()
    imfs_by_sensor = {}

    for sensor_num in selected_sensors:
        # For left foot sensors
        sensor_name_L = f'VGRF_L{sensor_num}'
        signal_L = df[sensor_name_L].values
        imfs_L = emd(signal_L)
        for i, imf in enumerate(imfs_L):
            imfs_by_sensor[f'{sensor_name_L}_IMF{i+1}'] = imf

        # For right foot sensors
        sensor_name_R = f'VGRF_R{sensor_num}'
        signal_R = df[sensor_name_R].values
        imfs_R = emd(signal_R)
        for i, imf in enumerate(imfs_R):
            imfs_by_sensor[f'{sensor_name_R}_IMF{i+1}'] = imf

    return imfs_by_sensor


imfs_by_sensor = find_imfs(df)
# Power spectral analysis of the IMFs using scipy.signal
def calculate_psd(imfs):
    psd_dict = {}  
    fs = 100 # sampling frequency in Hz

    for i, imf in enumerate(imfs): # imfs being 2D array & each row an IMF
        f, Pxx_den = scipy.signal.periodogram(imf, fs)
        psd_dict[f'IMF{i+1}'] = (f, Pxx_den)

    return psd_dict


In [43]:
imfs_df = pd.DataFrame(list(imfs_by_sensor.items()), columns=['sensor_name', 'IMFs'])
imfs_df.to_csv('IMF_GaCo16_01.csv', index=False)

In [54]:
print(imfs_df['IMFs'][0].shape)

(12119,)


In [25]:
def plot_imfs(signal, imfs, sensor_name):
    """
    Plots the original signal and its corresponding IMFs.
    
    :param signal: The original signal (1D numpy array).
    :param imfs: The Intrinsic Mode Functions (2D numpy array where each row is an IMF).
    :param sensor_name: The name of the sensor (string) for titling the plot.
    """
    n_imfs = imfs.shape[0]
    
    # Create a figure with subplots
    fig, axs = plt.subplots(n_imfs + 1, 1, figsize=(10, 2 * n_imfs))  # +1 for the original signal
    
    # Plot the original signal
    axs[0].plot(signal)
    axs[0].set_title(f'Original Signal: {sensor_name}')
    axs[0].set_ylabel('Amplitude')
    
    # Plot each IMF in a separate subplot
    for i in range(n_imfs):
        axs[i + 1].plot(imfs[i])
        axs[i + 1].set_title(f'IMF {i+1}')
        axs[i + 1].set_ylabel('Amplitude')
    
    # Set the xlabel for the last subplot
    axs[-1].set_xlabel('Time')

    plt.tight_layout()
    plt.show()

# Example usage for plotting the IMFs for a specific sensor (e.g., 'VGRF_L8')
sensor_name = 'VGRF_L8'
signal = df[sensor_name].values  # Make sure df and sensor_name are defined

imfs = imfs_df[sensor_name]  # Make sure imfs_by_sensor is defined and contains the IMFs

plot_imfs(signal, imfs_by_sensor, sensor_name)

KeyError: 'VGRF_L8'

In [31]:
# ploting function for psd_dict

def plot_psd_for_all_imfs(psd_dict):
    plt.figure(figsize=(10, 6))
    for imf, (freqs, pxx_den) in psd_dict.items():
        plt.plot(freqs, pxx_den, label=imf)
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.title('Power Spectral Density for Each IMF')
    plt.legend()
    plt.show()

# print(imfs)

In [32]:
psd_dict = calculate_psd(imfs)
plot_psd_for_all_imfs(psd_dict)

NameError: name 'imfs' is not defined

In [161]:
# Extracting dominant IMFs using psd done

def find_dominant_imfs(psd_dict, N):
    total_power = {imf: np.sum(pxx_den) for imf, (f,pxx_den) in psd_dict.items()}

    sorted_imfs = sorted(total_power.items(), key=lambda x:x[1], reverse=True)
    # Select top N
    dominant_imfs = sorted_imfs[:N]

    return dict(dominant_imfs)

In [162]:
# dominant_imfs = find_dominant_imfs(psd_dict,3)
# Ensure this returns the expected dominant IMFs
# dominant_imfs_keys = find_dominant_imfs(psd_dict, n=3)
# print("Dominant IMFs:", dominant_imfs)

def plot_dominant_imfs(psd_dict, dominant_imfs_keys):
    plt.figure(figsize=(10, 6))
    
    for imf in psd_dict:
        if imf not in dominant_imfs_keys.keys():
            freqs, pxx_den = psd_dict[imf]
            plt.plot(freqs, pxx_den, linestyle='--', alpha=0.5, label=imf)
    
    # Now plot the dominant IMFs for clear visibility
    for imf in dominant_imfs_keys.keys():
        freqs, pxx_den = psd_dict[imf]
        plt.plot(freqs, pxx_den, linewidth=2, marker='o', markersize=4, label=imf)
    
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.title('Power Spectral Density for Dominant IMFs')
    plt.legend()
    plt.show()

# plot_dominant_imfs(psd_dict, dominant_imfs)

In [163]:
def find_imfs(df, T, file_name):
    selected_sensors = [2, 4, 6, 8]
    emd = EMD()
    imfs_by_sensor = {}
    f = 100
    start_index = 0
    end_index = len(df['VGRF_L1'])

    total_points = len(df['VGRF_L1'])

    if total_points < 12000:
        print(f"Length of {df.name} is less than 2 min: {total_points}", file=file_name)
    else:
        mid_point = total_points // 2
        start_index = mid_point - 5000 
        end_index = mid_point + 5000

        # Print the length of the original column

    for sensor_num in selected_sensors:
        sensor_name_L = f'VGRF_L{sensor_num}'
        sensor_name_R = f'VGRF_R{sensor_num}'


        # Extract the middle 100 seconds of data
        data_L = df[sensor_name_L].values[start_index:end_index]
        data_R = df[sensor_name_R].values[start_index:end_index]

        # Apply EMD and round the IMFs to two decimal places
        imfs_L = np.round(emd(data_L), 2)
        imfs_R = np.round(emd(data_R), 2)

        # Print the length of the generated array

        imfs_by_sensor[sensor_name_L] = imfs_L
        imfs_by_sensor[sensor_name_R] = imfs_R
    
    print(f"Length of IMF for {df.name} is: {imfs_L.size}", file=file_name)

    return imfs_by_sensor


def find_imfs_no_log(df, T):
    selected_sensors = [2, 4, 6, 8]
    emd = EMD()
    imfs_by_sensor = {}
    f = 100
    start_index = 0
    end_index = len(df['VGRF_L1'])

    total_points = len(df['VGRF_L1'])

    # if total_points < 12000:
    #     # print(f"Length of {df.name} is less than 2 min: {total_points}", file=file_name)
    # else:
    #     mid_point = total_points // 2
    #     start_index = mid_point - 5000 
    #     end_index = mid_point + 5000

        # Print the length of the original column

    for sensor_num in selected_sensors:
        sensor_name_L = f'VGRF_L{sensor_num}'
        sensor_name_R = f'VGRF_R{sensor_num}'


        # Extract the middle 100 seconds of data
        data_L = df[sensor_name_L].values[start_index:end_index]
        data_R = df[sensor_name_R].values[start_index:end_index]

        # Apply EMD and round the IMFs to two decimal places
        imfs_L = np.round(emd(data_L), 2)
        imfs_R = np.round(emd(data_R), 2)

        # Print the length of the generated array

        imfs_by_sensor[sensor_name_L] = imfs_L
        imfs_by_sensor[sensor_name_R] = imfs_R
    
    # print(f"Length of IMF for {df.name} is: {imfs_L.size}", file=file_name)

    return imfs_by_sensor


def calculate_psd(imfs):
    psd_dict = {}
    fs = 100  # Sampling frequency in Hz

    for i, imf in enumerate(imfs):
        f, Pxx_den = scipy.signal.periodogram(imf, fs)
        psd_dict[f'IMF{i + 1}'] = (f, Pxx_den)

    return psd_dict

def process_individual_file(df, T, num_dominant_IMFs, num_sensors, log_file):
    selected_sensors = [2, 4, 6, 8]
    imfs_individual = find_imfs(df, T, log_file)
    imfs_individual_selected = {}

    for sensor_index in selected_sensors:
        sensor_name_L = f'VGRF_L{sensor_index}'
        sensor_name_R = f'VGRF_R{sensor_index}'

        imfs_L = imfs_individual[sensor_name_L]
        imfs_R = imfs_individual[sensor_name_R]

        psd_L = calculate_psd(imfs_L)
        psd_R = calculate_psd(imfs_R)
        dominant_imfs_L_dict = find_dominant_imfs(psd_L, num_dominant_IMFs)
        dominant_imfs_R_dict = find_dominant_imfs(psd_R, num_dominant_IMFs)

        for i, (key, _) in enumerate(dominant_imfs_L_dict.items()):
            imfs_individual_selected[f'{sensor_name_L}_IMF{i + 1}'] = np.round(imfs_L[int(key.replace('IMF', '')) - 1], 2)
        for i, (key, _) in enumerate(dominant_imfs_R_dict.items()):
            imfs_individual_selected[f'{sensor_name_R}_IMF{i + 1}'] = np.round(imfs_R[int(key.replace('IMF', '')) - 1], 2)

    return imfs_individual_selected

def process_individual_file_no_log(df, T, num_dominant_IMFs, num_sensors):
    selected_sensors = [2, 4, 6, 8]
    imfs_individual = find_imfs_no_log(df, T)
    imfs_individual_selected = {}

    for sensor_index in selected_sensors:
        sensor_name_L = f'VGRF_L{sensor_index}'
        sensor_name_R = f'VGRF_R{sensor_index}'

        imfs_L = imfs_individual[sensor_name_L]
        imfs_R = imfs_individual[sensor_name_R]

        psd_L = calculate_psd(imfs_L)
        psd_R = calculate_psd(imfs_R)
        dominant_imfs_L_dict = find_dominant_imfs(psd_L, num_dominant_IMFs)
        dominant_imfs_R_dict = find_dominant_imfs(psd_R, num_dominant_IMFs)

        for i, (key, _) in enumerate(dominant_imfs_L_dict.items()):
            imfs_individual_selected[f'{sensor_name_L}_IMF{i + 1}'] = np.round(imfs_L[int(key.replace('IMF', '')) - 1], 2)
        for i, (key, _) in enumerate(dominant_imfs_R_dict.items()):
            imfs_individual_selected[f'{sensor_name_R}_IMF{i + 1}'] = np.round(imfs_R[int(key.replace('IMF', '')) - 1], 2)

    return imfs_individual_selected

# dataset_dir_control = "csv_averaged_force_data_4col/GaCo04_01.csv"
# dataset_dir_pd = "csv_averaged_force_data_4col/GaPt03_01.csv"

In [47]:

# Define your data_files, data_directory_Ju, and features variables here
data_files = [f for f in os.listdir(data_directory_Ju) if os.path.isfile(os.path.join(data_directory_Ju, f)) and f.endswith('01.txt')]
features = ['Time','VGRF_L1','VGRF_L2','VGRF_L3','VGRF_L4','VGRF_L5','VGRF_L6','VGRF_L7','VGRF_L8',
            'VGRF_R1','VGRF_R2','VGRF_R3','VGRF_R4','VGRF_R5','VGRF_R6','VGRF_R7','VGRF_R8','Total Force_L','Total Force_R']

file_error = []
output_directory = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Output_IMF"
log_file_path = os.path.join(output_directory, "processing_log.txt")  # Path for the log file


# Open the log file for writing
with open(log_file_path, "w") as log_file:
    for file_name in data_files:
        if file_name.startswith('.'):
            # Skip hidden files like .DS_Store
            continue

        print(f"Processing file: {file_name}")
        file_path = os.path.join(data_directory_Ju, file_name)

        try:
            df = pd.read_csv(file_path, delim_whitespace=True, header=None, encoding_errors='ignore')
            df.columns = features
            df.name = file_name.split('\\')[-1]

            T = 110
            num_dominant_IMFs = 3  # Adjust as needed
            num_sensors = 4  # Adjust as needed
            dominant_imfs = process_individual_file(df, T, num_dominant_IMFs, num_sensors, log_file)
            
            imfs_df = pd.DataFrame(list(dominant_imfs.items()), columns=['sensor_name', 'IMFs'])
            imfs_df.describe()
            output_file_path = os.path.join(output_directory, f"{file_name.split('.')[0]}_dominant_IMFs.csv")
            imfs_df.to_csv(output_file_path, index=False)
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            file_error.append(file_path)

    if file_error:
        print("Files with errors:", file_error, file=log_file)
    else:
        print("All files processed successfully.", file=log_file)


Processing file: JuCo23_01.txt
Processing file: JuPt12_01.txt
Processing file: JuCo07_01.txt
Processing file: JuCo19_01.txt
Processing file: JuPt24_01.txt
Processing file: JuPt28_01.txt
Processing file: JuCo15_01.txt
Processing file: JuPt10_01.txt
Processing file: JuPt02_01.txt
Processing file: JuCo21_01.txt
Processing file: JuCo09_01.txt
Processing file: JuCo17_01.txt
Processing file: JuPt26_01.txt
Processing file: JuCo05_01.txt
Processing file: JuPt22_01.txt
Processing file: JuCo01_01.txt
Processing file: JuCo13_01.txt
Processing file: JuPt06_01.txt
Processing file: JuPt18_01.txt
Processing file: JuCo25_01.txt
Processing file: JuPt14_01.txt
Processing file: JuCo11_01.txt
Processing file: JuCo03_01.txt
Processing file: JuPt20_01.txt
Processing file: JuPt08_01.txt
Processing file: JuPt16_01.txt
Processing file: JuPt04_01.txt
Processing file: JuCo18_01.txt
Processing file: JuPt25_01.txt
Processing file: JuCo06_01.txt
Processing file: JuPt29_01.txt
Processing file: JuCo14_01.txt
Processi

In [201]:
## Readjusting the process above for another useful approach 
## for consolidating data in a pandas dataframe

data_files = [f for f in os.listdir(data_directory_Ju) if os.path.isfile(os.path.join(data_directory_Ju, f)) and f.endswith('01.txt')]
features = ['Time', 'VGRF_L1', 'VGRF_L2', 'VGRF_L3', 'VGRF_L4', 'VGRF_L5', 'VGRF_L6', 'VGRF_L7', 'VGRF_L8',
            'VGRF_R1', 'VGRF_R2', 'VGRF_R3', 'VGRF_R4', 'VGRF_R5', 'VGRF_R6', 'VGRF_R7', 'VGRF_R8', 'Total Force_L', 'Total Force_R']

# all_data_df = pd.DataFrame()  # Initialize an empty DataFrame to store all data
all_data_df = pd.DataFrame()
file_error = []


for file_name in data_files[0:6]:
    if file_name.startswith('.'):
            # Skip hidden files like .DS_Store
            continue

    print(f"Processing file: {file_name}")
        
    file_path = os.path.join(data_directory_Ju, file_name)
    

    try:
        df = pd.read_csv(file_path, delim_whitespace=True, header=None, encoding_errors='ignore')
        df.columns = features
        df.name = file_name.split('\\')[-1]

        T = 6000 # 1 min of data expected minimum.
        num_dominant_IMFs = 3  # Adjust as needed
        num_sensors = 4  # Adjust as needed
        dominant_imfs = process_individual_file_no_log(df, T, num_dominant_IMFs, num_sensors)
        
        imfs_df = pd.DataFrame(list(dominant_imfs.items()), columns=['sensor_name', 'IMFs'])
        # Assuming it returns a DataFrame or Series that you want to append
        # processed_data = process_individual_file(df, T, num_dominant_IMFs, num_sensors)
        if 'Pt' in file_name:
            imfs_df['Health'] = 1
        elif 'Co' in file_name:
            imfs_df['Health'] = 0

        # Assuming processed_data is a DataFrame or Series that can be appended directly
        all_data_df = pd.concat([all_data_df, imfs_df], ignore_index=True)
        
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")
        file_error.append(file_path)

    


# Now `all_data_df` contains all the processed data from each file
print("All data has been processed and combined into a single DataFrame.")

# Optionally, if you want to save the final DataFrame:
# all_data_df.to_csv('path/to/save/final_combined_data.csv', index=False)

if file_error:
    print("Files with errors:", file_error)
else:
    print("All files processed successfully.")

print(all_data_df)

Processing file: JuCo23_01.txt
Processing file: JuPt12_01.txt
Processing file: JuCo07_01.txt
Processing file: JuCo19_01.txt
Processing file: JuPt24_01.txt
Processing file: JuPt28_01.txt
All data has been processed and combined into a single DataFrame.
All files processed successfully.
      sensor_name                                               IMFs  Health
0    VGRF_L2_IMF1  [0.86, 1.77, 2.49, 3.03, 3.42, 3.65, 3.75, 3.7...       0
1    VGRF_L2_IMF2  [438.09, 434.24, 430.31, 426.29, 422.19, 418.0...       0
2    VGRF_L2_IMF3  [86.68, 86.92, 86.27, 84.76, 82.38, 79.17, 75....       0
3    VGRF_R2_IMF1  [-97.88, -100.2, -102.44, -104.62, -106.73, -1...       0
4    VGRF_R2_IMF2  [-77.52, -78.04, -78.39, -78.45, -78.06, -77.0...       0
..            ...                                                ...     ...
139  VGRF_L8_IMF2  [-0.1, -0.29, -0.48, -0.65, -0.82, -0.97, -1.1...       1
140  VGRF_L8_IMF3  [0.28, 0.49, 0.68, 0.86, 1.03, 1.17, 1.29, 1.3...       1
141  VGRF_R8_IMF1  [-

In [210]:
# print(type(all_data_df.iloc[10,1])) 
print(all_data_df.iloc[24])

sensor_name                                         VGRF_L2_IMF1
IMFs           [47.97, 51.2, 54.4, 57.58, 60.71, 63.79, 66.82...
Health                                                         1
Name: 24, dtype: object


In [226]:
def extract_features(all_data_df):
    # Simple feature extraction: Mean, STD, and Max for simplicity
    features = {}
    for i, row in all_data_df.iterrows():
    
        imf_data = row['IMFs']  # Converting string list to actual list
        # print(imf_data)
        features[f'{row["sensor_name"]}_mean'] = np.mean(imf_data)
        # print(features)
        features[f'{row["sensor_name"]}_std'] = np.std(imf_data)
        features[f'{row["sensor_name"]}_max'] = np.max(imf_data)
    return pd.Series(features)

num_sensors_per_individual = 24
feature_df = pd.DataFrame()

for start in range(0, all_data_df.shape[0], num_sensors_per_individual):
    end = start + num_sensors_per_individual
    individual_df = all_data_df.iloc[start:end]
    features = extract_features(individual_df)
    feature_df = pd.concat([feature_df, features], ignore_index=True)
    


[   0.86    1.77    2.49 ... -151.82 -150.78 -149.34]
[438.09 434.24 430.31 ... 162.01 161.99 161.91]
[86.68 86.92 86.27 ...  7.43 -0.59 -8.52]
[ -97.88 -100.2  -102.44 ...  -72.17  -69.84  -67.48]
[-77.52 -78.04 -78.39 ... -33.2  -33.16 -33.13]
[25.77 27.84 29.94 ... -3.58 -3.98 -4.37]
[-77.2  -76.95 -76.7  ...  91.38  92.89  94.34]
[114.68 114.29 113.93 ... -24.25 -24.88 -25.52]
[ 1.000e-01 -4.000e-02 -9.000e-02 ...  3.488e+01  3.908e+01  4.299e+01]
[-120.48 -120.87 -121.24 ... -158.95 -158.31 -157.64]
[-31.28 -32.74 -33.8  ...  87.1   86.46  85.76]
[ 74.    74.13  74.26 ... -32.13 -32.1  -32.08]
[ 72.92  73.02  73.05 ... 113.39 115.58 117.68]
[-1.48 -0.49  0.47 ... 25.29 27.09 28.59]
[ 1.75  1.06  0.39 ... -0.48 -0.56 -0.56]
[-112.87 -112.1  -111.23 ... -118.89 -117.81 -116.83]
[ 4.48  3.85  3.23 ... 88.84 88.7  88.52]
[ 59.44  59.71  59.77 ... -11.79 -12.22 -12.3 ]
[ 3.99  3.9   3.77 ... -0.16 -0.14 -0.06]
[-6.71 -6.22 -5.69 ... -0.14 -0.17 -0.2 ]
[ 1.87  1.74  1.6  ... -4.25 -3.93

In [53]:
## For Ga Study 
# Define your data_files, data_directory_Ju, and features variables here
data_files = [f for f in os.listdir(data_directory_Ga) if os.path.isfile(os.path.join(data_directory_Ga, f)) and f.endswith('01.txt')]
features = ['Time','VGRF_L1','VGRF_L2','VGRF_L3','VGRF_L4','VGRF_L5','VGRF_L6','VGRF_L7','VGRF_L8',
            'VGRF_R1','VGRF_R2','VGRF_R3','VGRF_R4','VGRF_R5','VGRF_R6','VGRF_R7','VGRF_R8','Total Force_L','Total Force_R']

file_error = []
output_directory = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Output_IMF_Ga"
log_file_path = os.path.join(output_directory, "processing_log.txt")  # Path for the log file

# Open the log file for writing
with open(log_file_path, "w") as log_file:
    for file_name in data_files:
        if file_name.startswith('.'):
            # Skip hidden files like .DS_Store
            continue

        print(f"Processing file: {file_name}")
        file_path = os.path.join(data_directory_Ga, file_name)

        try:
            df = pd.read_csv(file_path, delim_whitespace=True, header=None, encoding_errors='ignore')
            df.columns = features
            df.name = file_name.split('\\')[-1]

            T = 110
            num_dominant_IMFs = 3  # Adjust as needed
            num_sensors = 4  # Adjust as needed
            dominant_imfs = process_individual_file(df, T, num_dominant_IMFs, num_sensors, log_file)
            
            imfs_df = pd.DataFrame(list(dominant_imfs.items()), columns=['sensor_name', 'IMFs'])
            imfs_df.describe()
            output_file_path = os.path.join(output_directory, f"{file_name.split('.')[0]}_dominant_IMFs.csv")
            imfs_df.to_csv(output_file_path, index=False)
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            file_error.append(file_path)

    if file_error:
        print("Files with errors:", file_error, file=log_file)
    else:
        print("All files processed successfully.", file=log_file)


Processing file: GaPt33_01.txt
Processing file: GaCo10_01.txt
Processing file: GaPt21_01.txt
Processing file: GaCo02_01.txt
Processing file: GaPt17_01.txt
Processing file: GaPt09_01.txt
Processing file: GaPt05_01.txt
Processing file: GaPt23_01.txt
Processing file: GaCo12_01.txt
Processing file: GaPt31_01.txt
Processing file: GaPt19_01.txt
Processing file: GaPt07_01.txt
Processing file: GaPt15_01.txt
Processing file: GaPt03_01.txt
Processing file: GaCo16_01.txt
Processing file: GaCo08_01.txt
Processing file: GaCo04_01.txt
Processing file: GaPt27_01.txt
Processing file: GaCo22_01.txt
Processing file: GaPt13_01.txt
Processing file: GaPt25_01.txt
Processing file: GaCo06_01.txt
Processing file: GaPt29_01.txt
Processing file: GaCo14_01.txt
Processing file: GaPt08_01.txt
Processing file: GaPt16_01.txt
Processing file: GaPt04_01.txt
Processing file: GaCo11_01.txt
Processing file: GaPt32_01.txt
Processing file: GaCo03_01.txt
Processing file: GaPt20_01.txt
Processing file: GaPt06_01.txt
Processi

In [54]:
# Define your data_files, data_directory_Ju, and features variables here
data_files = [f for f in os.listdir(data_directory_Si) if os.path.isfile(os.path.join(data_directory_Si, f)) and f.endswith('01.txt')]
features = ['Time','VGRF_L1','VGRF_L2','VGRF_L3','VGRF_L4','VGRF_L5','VGRF_L6','VGRF_L7','VGRF_L8',
            'VGRF_R1','VGRF_R2','VGRF_R3','VGRF_R4','VGRF_R5','VGRF_R6','VGRF_R7','VGRF_R8','Total Force_L','Total Force_R']

file_error = []
output_directory = "/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Output_IMF_Si"
log_file_path = os.path.join(output_directory, "processing_log.txt")  # Path for the log file

# Open the log file for writing
with open(log_file_path, "w") as log_file:
    for file_name in data_files:
        if file_name.startswith('.'):
            # Skip hidden files like .DS_Store
            continue

        print(f"Processing file: {file_name}")
        file_path = os.path.join(data_directory_Si, file_name)

        try:
            df = pd.read_csv(file_path, delim_whitespace=True, header=None, encoding_errors='ignore')
            df.columns = features
            df.name = file_name.split('\\')[-1]

            T = 110
            num_dominant_IMFs = 3  # Adjust as needed
            num_sensors = 4  # Adjust as needed
            dominant_imfs = process_individual_file(df, T, num_dominant_IMFs, num_sensors, log_file)
            
            imfs_df = pd.DataFrame(list(dominant_imfs.items()), columns=['sensor_name', 'IMFs'])
            imfs_df.describe()
            output_file_path = os.path.join(output_directory, f"{file_name.split('.')[0]}_dominant_IMFs.csv")
            imfs_df.to_csv(output_file_path, index=False)
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            file_error.append(file_path)

    if file_error:
        print("Files with errors:", file_error, file=log_file)
    else:
        print("All files processed successfully.", file=log_file)


Processing file: SiPt12_01.txt
Processing file: SiCo23_01.txt
Processing file: SiPt36_01.txt
Processing file: SiPt28_01.txt
Processing file: SiCo15_01.txt
Processing file: SiCo19_01.txt
Processing file: SiPt24_01.txt
Processing file: SiCo07_01.txt
Processing file: SiCo21_01.txt
Processing file: SiPt02_01.txt
Processing file: SiPt10_01.txt
Processing file: SiCo05_01.txt
Processing file: SiPt38_01.txt
Processing file: SiCo17_01.txt
Processing file: SiPt34_01.txt
Processing file: SiCo09_01.txt
Processing file: SiCo13_01.txt
Processing file: SiPt30_01.txt
Processing file: SiCo01_01.txt
Processing file: SiPt22_01.txt
Processing file: SiCo29_01.txt
Processing file: SiPt14_01.txt
Processing file: SiPt18_01.txt
Processing file: SiCo25_01.txt
Processing file: SiPt20_01.txt
Processing file: SiCo03_01.txt
Processing file: SiPt32_01.txt
Processing file: SiCo11_01.txt
Processing file: SiPt04_01.txt
Processing file: SiCo27_01.txt
Processing file: SiPt16_01.txt
Processing file: SiPt08_01.txt
Processi

In [177]:
## Now going into the preparation of all csv data for use as training inputs


file_path_output = '/Users/toufikjrab/Projects/gait-in-parkinsons-disease-1.0.0/Output_IMF_Ga'
data_files_output = [f for f in os.listdir(file_path_output) if f.endswith('.csv') and not f.startswith('.')]

def convert_to_array(str_arr):
    if pd.isna(str_arr):
        return np.array([])
    try:
        # Remove the brackets and split the string
        str_arr = str_arr.strip('[]')
        # Skip over the ellipsis if present
        if '...' in str_arr:
            # You might want to log this or handle differently because this means losing data
            print(f"Skipping conversion due to ellipsis in data: {str_arr}")
            return np.array([])
        # Convert to float and create numpy array
        return np.array([float(item) for item in str_arr.split()])
    except Exception as e:
        print(f"Error converting: {str_arr} with error {e}")
        return np.array([])


all_individuals_data = [] 
all_individuals_labels = []

for file in data_files_output:
    
    file_path = os.path.join(file_path_output, file)

    df = pd.read_csv(file_path)
    # print(type(df.iloc[10,1])) 

    # df['IMFs'] = df['IMFs'].apply(convert_to_array)
    
    # max_length = df.applymap(len).max().max()
    
    flattened_data = df.values.flatten()
    all_individuals_data.append(flattened_data)

    if 'Pt' in file:
        all_individuals_labels.append('PD')
    elif 'Co' in file:
        all_individuals_labels.append('Healthy')

    
    # print(f"Shape of individual {file} data: {flattened_data.shape}")
    

combined_data = np.vstack(all_individuals_data)

combined_df = pd.DataFrame(combined_data)

num_individuals = combined_df.shape[0]
num_time_steps = combined_df.shape[1] // (3*8)
num_features = 3 * 8 # 3 IMFs for each of 8 sensors

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(all_individuals_labels)

def convert_to_array(str_arr):
    return np.array(list(map(float, str_arr.strip('[]').split())))

# for col in combined_df.columns:
#     if col.startswith('VGRF'):
#         combined_df[col] = combined_df[col].apply(convert_to_array)


print(combined_data)

max_length = 12000

# from keras.preprocessing.sequence import pad_sequences

# for col in combined_df[1]:
#     if col.startwith('VGRF'):

[['VGRF_L2_IMF1' '[ 31.05  32.06  31.56 ... 174.78 172.82 170.48]'
  'VGRF_L2_IMF2' ... '[-23.28 -23.4  -23.53 ... -49.96 -50.99 -51.29]'
  'VGRF_R8_IMF3' '[-0.01 -0.17 -0.13 ... -6.5  -6.12 -5.81]']
 ['VGRF_L2_IMF1' '[27.34 27.76 28.09 ... 15.63 13.68 11.75]'
  'VGRF_L2_IMF2' ... '[ -5.38  -2.85  -0.5  ... -13.72 -12.8  -11.74]'
  'VGRF_R8_IMF3' '[ -1.15  -0.23   0.62 ... -16.59 -16.15 -15.77]']
 ['VGRF_L2_IMF1' '[-21.7  -26.88 -32.44 ... -22.46 -18.76 -14.66]'
  'VGRF_L2_IMF2' ... '[42.47 44.32 46.14 ... 30.18 29.38 28.8 ]'
  'VGRF_R8_IMF3' '[ -9.27  -9.59 -10.74 ... -45.07 -44.54 -43.94]']
 ...
 ['VGRF_L2_IMF1' '[ 67.49  65.89  64.24 ... -53.38 -53.96 -54.5 ]'
  'VGRF_L2_IMF2' ... '[  3.27   3.31   3.37 ... -11.16 -10.27  -9.4 ]'
  'VGRF_R8_IMF3' '[ -0.73  -0.76  -0.78 ... -27.93 -27.85 -27.7 ]']
 ['VGRF_L2_IMF1' '[137.67 135.59 133.37 ...  53.78  45.67  37.3 ]'
  'VGRF_L2_IMF2' ... '[ 10.81  10.32   9.71 ... -35.23 -35.13 -34.58]'
  'VGRF_R8_IMF3' '[-30.79 -30.77 -30.72 ... -40.9  

In [178]:
## Splitting the data

X_train, X_test, y_train, y_test = train_test_split(combined_df, y_encoded, test_size=0.2, random_state=42)

In [179]:
## NOW ONTO ML TRAINING 

# Define model architecture - based on StackOverFlow
# Link: https://stackoverflow.com/questions/55433649/how-to-combine-lstm-and-cnn-models-in-keras


num_time_steps = 6000
num_features = 24

# For explanation of the architecture to its parameters, I (Toufic) am adding detailed comments

model = Sequential() # linear stack of layers in Keras (the API used by TensorFlow )

## Now onto the CNN:

# Firstly comes the 1D convolutional layer
# Filters = 64 as number of outputs filters in the convolution, kernel size=3 of 1D convolution window
# Activation function used per the paper is rectified linear unit
model.add(Conv1D(filters=64, kernel_size = 3, activation = 'relu',input_shape= (num_time_steps, num_features)))

# This layer is for downsampling the input along the time dimensions (time series)
# To detect invariant features with a pooling window size of 2
model.add(MaxPooling1D(pool_size=2))

# Regularization common to prevent overfitting
model.add(Dropout(0.5))

# Another 1D convolution layer, also commonly used to stack convolutional and pooling layers
# to allow the model to learn more complex features
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

## Flatten output from CNN layers before passing onto LSTM
## to make the output of the convolutional layers suitable for the LSTM layer

timesteps_after_conv = num_time_steps // 4  # Adjust depending on your pooling layers
features_after_conv = 64 # This assumes no padding in Conv layers
# total_features_after_flattening = timesteps_after_conv * features_after_conv

# model.add(Flatten())

# if total_features_after_flattening != timesteps_after_conv * 128:
#     raise ValueError(f"Not expected dimensions. Expected {total_features_after_flattening} but got {timesteps_after_conv*128}")

# model.add(Reshape((timesteps_after_conv, features_after_conv)))

# LSTM unmber of units or layers, these can remember long dependencies in sequences of data
model.add(LSTM(100))
model.add(Dropout(0.5))

## Per the paper, they used softmax LSTM, Adam optimizer, and a cross_entropy loss function

# This fully connected layer returns an output for each class, here PD or healthy so 2
model.add(Dense(2, activation='softmax'))  # Assuming a classification task


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [180]:
# model.fit(X_train,y_train, validation_batch_size=(X_test, y_test), epochs =100)
print(X_train)
print(y_train)

              0                                                  1   \
8   VGRF_L2_IMF1    [-62.   -64.45 -66.59 ... 109.44 109.64 109.48]   
3   VGRF_L2_IMF1    [-15.31 -17.06 -18.76 ...  23.81  23.06  22.39]   
6   VGRF_L2_IMF1    [ 26.52  25.92  25.07 ... -89.03 -87.85 -86.51]   
40  VGRF_L2_IMF1  [ 3.3422e+02  3.3166e+02  3.2874e+02 ... -2.20...   
33  VGRF_L2_IMF1    [ 52.07  50.81  49.36 ... -92.35 -91.78 -91.2 ]   
13  VGRF_L2_IMF1    [  4.65   2.24  -0.5  ...  16.5    3.57 -11.32]   
17  VGRF_L2_IMF1    [  6.17  -5.25 -16.34 ...  53.05  54.01  54.5 ]   
44  VGRF_L2_IMF1    [ 67.49  65.89  64.24 ... -53.38 -53.96 -54.5 ]   
15  VGRF_L2_IMF1    [-91.81 -92.39 -92.76 ...  11.54  13.07  14.16]   
9   VGRF_L2_IMF1    [-33.99 -36.21 -38.33 ...  30.33  30.28  30.19]   
16  VGRF_L2_IMF1  [  69.45   70.51   71.34 ... -115.   -114.47 -...   
29  VGRF_L2_IMF1    [  5.49   4.65   3.81 ... -83.32 -82.75 -82.01]   
32  VGRF_L2_IMF1    [-77.35 -81.74 -87.79 ...  95.39 100.19 104.52]   
45  VG

In [120]:
model.fit(X_train,y_train, validation_batch_size=(X_test, y_test), epochs =100)


Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_21_1/Cast:0", shape=(None, 48), dtype=float32). Expected shape (None, 10000, 24), but input has incompatible shape (None, 48)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 48), dtype=string)
  • training=True
  • mask=None

In [ ]:


merged = Concatenate()([model1.output,model2.output])
z = Dense(128, activation="relu")(merged)
z = Dropout(0.25)(z)
z = Dense(1024, activation="relu")(z)
z = Dense(1, activation="sigmoid")(z)

model = Model(inputs=[model1.input, model2.input], outputs=z)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit([x_train[train_index][:,:66], x_train[train_index][:,66:132], y_train[train_index], batch_size=100, epochs=100, verbose=2)